## Bechmark modelling - Linear models

The purpose of this file is to create model benchmarks which we will compare against the final optimized fits, this ensures we can  
check whether the optimization was successful and signifficant as well as changes in attribute weights (e.g. regression analysis)

The second part of the file is to perform regression analysis utilizing the benchmark models to determine the biggest contributors (attributes)  
to the target attribute = popularity.

In [3]:
import os
import sys

sys.dont_write_bytecode = True

import numpy as np
import pandas as pd

from dotenv import load_dotenv
load_dotenv()

False

In [50]:
from etl import *

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, root_mean_squared_error

loading data

In [12]:
DATA_DIR = './Data/'
DATA_FILE = 'spotify_tracks_kaggle_weekly.csv'

In [13]:
data = pd.read_csv(DATA_DIR + DATA_FILE)

data split on predefined conditions

In [20]:
RANDOM_STATE = 21
TEST_SIZE = 0.1

In [21]:
X = data.drop('popularity', axis=1)
y = data['popularity']

In [22]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE
)

### Testing scenario

We will compare the transformation pipeline results against the basic dataset form, to determine the utility of the transformations.  
It is important to state that the overall benefit of transformation pipeline could be present after the optimization - that means the verdict of the effectiveness  
is valid only for the benchmark linear fits.

#### 1. Original data - Nulls are not processed, original encoding, not-scaled

In [123]:
models_1 = {
    'linear_regression' : LinearRegression(),
    'Ridge' : Ridge(),
    'Lasso' : Lasso()
}

In [124]:
drop_columns = ['track_id', 'artwork_url', 'track_url',
                 'track_name', 'album_name', 'artist_name', 'language']

In [125]:
X_train = X_train.drop(drop_columns, axis=1, errors='ignore')
X_test = X_test.drop(drop_columns, axis=1, errors='ignore')

In [126]:
for model_name, model in models_1.items():
    model.fit(X_train, y_train)
    print(f"fitting {model_name:<20} ... train R^2 = {model.score(X_train, y_train):.5f}")

fitting linear_regression    ... train R^2 = 0.05611
fitting Ridge                ... train R^2 = 0.05611
fitting Lasso                ... train R^2 = 0.00268


prediction

In [127]:
for model_name, model in models_1.items():
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = root_mean_squared_error(y_test, y_pred)

    print(f"Model: {model_name:<20} ... test R^2 = {model.score(X_test, y_test):.5f} | MSE: {mse:.5f} | RMSE: {rmse:.5f}")

Model: linear_regression    ... test R^2 = 0.05155 | MSE: 338.23823 | RMSE: 18.39125
Model: Ridge                ... test R^2 = 0.05156 | MSE: 338.23771 | RMSE: 18.39124
Model: Lasso                ... test R^2 = 0.00247 | MSE: 355.74171 | RMSE: 18.86112


#### 2. Transformed data - includes custom encoding, scaling, ...

In [128]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE
)

drop_columns = ['track_id', 'artwork_url', 'track_url', 'track_name', 'album_name']

X_train = X_train.drop(drop_columns, axis=1, errors='ignore')
X_test = X_test.drop(drop_columns, axis=1, errors='ignore')

In [129]:
target = 'popularity'

onehot_col = ['language']
circle_of_fifths_col = ['key']
artist_name_col = ['artist_name']

numeric_columns = list(X_train.columns[X_train.dtypes != object].difference(['key', 'mode']))

nan_columns = ['acousticness', 'danceability', 'energy', 'liveness', 'speechiness', 'tempo', 'valence']

In [130]:
numeric_pipeline = Pipeline(steps=[
    ('scaling', StandardScaler())
])

artist_name_pipeline = Pipeline(steps=[
    ('encoding', FrequencyEncoder()),
    ('scaling', StandardScaler())
])


transformations = ColumnTransformer(transformers=[
    
    ('onehot_encoding', OneHotEncoder(sparse_output=False), onehot_col),
    ('trigonometric_encoding', CircleOfFifthsEncoding(), circle_of_fifths_col),
    ('artist_encoding', artist_name_pipeline, artist_name_col),
    ('nummeric_processing', numeric_pipeline, numeric_columns)

], remainder='drop')


preprocessing = Pipeline(steps=[
    ('null_values', ConvertNull(columns=nan_columns)),
    ('transformation', transformations)
])

In [131]:
preprocessing.fit(X_train)

Pipeline(steps=[('null_values',
                 ConvertNull(columns=['acousticness', 'danceability', 'energy',
                                      'liveness', 'speechiness', 'tempo',
                                      'valence'])),
                ('transformation',
                 ColumnTransformer(transformers=[('onehot_encoding',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['language']),
                                                 ('trigonometric_encoding',
                                                  CircleOfFifthsEncoding(),
                                                  ['key']),
                                                 ('artist_encoding',
                                                  Pipeline(steps=[('encoding',
                                                                   FrequencyEncoder()),
                                                                  ('scaling',
                                                                   StandardScaler())]),
                                                  ['artist_name']),
                                                 ('nummeric_processing',
                                                  Pipeline(steps=[('scaling',
                                                                   StandardScaler())]),
                                                  ['acousticness',
                                                   'danceability',
                                                   'duration_ms', 'energy',
                                                   'instrumentalness',
                                                   'liveness', 'loudness',
                                                   'speechiness', 'tempo',
                                                   'time_signature', 'valence',
                                                   'year'])]))])

In [132]:
X_train_transformed = preprocessing.transform(X_train)
X_test_transformed = preprocessing.transform(X_test)

Nulls dropped

In [133]:
X_train_transformed = pd.DataFrame(X_train_transformed).dropna()
X_test_transformed = pd.DataFrame(X_test_transformed).dropna()

y_train_transformed = y_train.reset_index(drop=True).loc[X_train_transformed.index]
y_test_transformed = y_test.reset_index(drop=True).loc[X_test_transformed.index]

In [135]:
models_2 = {
    'linear_regression' : LinearRegression(),
    'Ridge' : Ridge(),
    'Lasso' : Lasso()
}

In [136]:
for model_name, model in models_2.items():
    model.fit(X_train_transformed, y_train_transformed)
    print(f"fitting {model_name:<20} ... train R^2 = {model.score(X_train_transformed, y_train_transformed):.5f}")

print()
print()

for model_name, model in models_2.items():
    y_pred = model.predict(X_test_transformed)
    mse = mean_squared_error(y_test_transformed, y_pred)
    rmse = root_mean_squared_error(y_test_transformed, y_pred)

    print(f"Model: {model_name:<20} ... test R^2 = {model.score(X_test_transformed, y_test_transformed):.5f} | MSE: {mse:.5f} | RMSE: {rmse:.5f}")

fitting linear_regression    ... train R^2 = 0.11218
fitting Ridge                ... train R^2 = 0.10617
fitting Lasso                ... train R^2 = 0.04652


Model: linear_regression    ... test R^2 = 0.11385 | MSE: 316.11583 | RMSE: 17.77965
Model: Ridge                ... test R^2 = 0.10924 | MSE: 317.76089 | RMSE: 17.82585
Model: Lasso                ... test R^2 = 0.04721 | MSE: 339.88902 | RMSE: 18.43608


Verdict: Transformation pipeline doubled the quality of the fit from 5% --> 10%

### Regression analysis

In [145]:
weights_1 = [model.coef_ for model in models_1.values()]
weights_2 = [model.coef_ for model in models_2.values()]

index = X_train_transformed.columns

In [157]:
results = pd.DataFrame({
    'model_name' : list(models_1.keys()),
    'weights_1' : weights_1,
    'weights_2' : weights_2
})